### **코드 설명**

- **문항 매개변수 저장**
  - `item_parameters` 데이터프레임을 생성하여 문항 번호, 변별도(α), 난이도(β)를 저장합니다.
  - `to_csv` 함수를 사용하여 `문항_매개변수.csv` 파일로 저장합니다.
  - `encoding='utf-8-sig'` 옵션을 사용하여 한글이 깨지지 않도록 합니다.

- **수험생 능력 추정치 저장**
  - 개인정보 보호를 위해 필요한 정보만 포함합니다. 여기서는 `이름`과 `능력 추정치`만 저장합니다.
  - 만약 수험생을 식별할 수 있는 민감한 정보(예: 주민등록번호 등)가 있다면 제외하거나 익명화해야 합니다.
  - `examinee_parameters` 데이터프레임을 생성하여 저장합니다.
  - `to_csv` 함수를 사용하여 `수험생_능력_추정치.csv` 파일로 저장합니다.

### **추가 사항**

- **개인정보 보호**
  - 데이터를 저장하거나 공유할 때는 개인정보 보호법 및 관련 규정을 준수해야 합니다.
  - 민감한 개인 식별 정보를 저장하거나 공유할 때는 반드시 적절한 동의를 받아야 하며, 필요에 따라 익명화 또는 비식별화 처리를 해야 합니다.

- **저장된 결과 활용**
  - 저장된 CSV 파일을 불러와서 분석하거나 보고서를 작성할 때 활용할 수 있습니다.
  - 예를 들어, Excel에서 파일을 열어 결과를 확인하거나, 다른 분석 프로그램에서 데이터를 불러올 수 있습니다.

### **CSV 파일 예시**

**문항_매개변수.csv**

| 문항 번호 | 변별도 (alpha) | 난이도 (beta) |
|-----------|----------------|---------------|
| 1         | 1.2345         | -0.5678       |
| 2         | 0.9876         | 0.1234        |
| ...       | ...            | ...           |

**수험생_능력_추정치.csv**

| 이름     | 능력 추정치 (theta) |
|----------|---------------------|
| 홍길동   | 0.4567              |
| 이영희   | -0.1234             |
| ...      | ...                 |

### **참고사항**

- **파일 경로 설정**
  - `to_csv` 함수에서 파일 경로를 지정하지 않으면 현재 작업 디렉토리에 파일이 저장됩니다.
  - 특정 폴더에 저장하고 싶다면 경로를 지정해 주세요. 예: `to_csv('results/문항_매개변수.csv', ...)`

- **다른 형식으로 저장**
  - 필요에 따라 Excel 파일(`.xlsx`)로 저장하거나, 데이터베이스에 저장할 수도 있습니다.
  - Excel로 저장하려면 `pandas`의 `to_excel` 함수를 사용할 수 있습니다.

    ```python
    item_parameters.to_excel('문항_매개변수.xlsx', index=False)
    examinee_parameters.to_excel('수험생_능력_추정치.xlsx', index=False)
    ```

- **추가 분석**
  - 저장된 결과를 활용하여 문항 분석, 수험생 능력 분포 분석 등 추가적인 분석을 수행할 수 있습니다.

### **요약**

- 추정된 문항 매개변수와 수험생의 능력 추정치를 CSV 파일로 저장하였습니다.
- 개인정보 보호를 위해 민감한 정보는 제외하고, 필요한 정보만 포함하였습니다.
- 저장된 결과는 추후 분석이나 보고서 작성 등에 활용할 수 있습니다.

---

In [6]:
import pandas as pd
import numpy as np
import pymc as pm

# Read the CSV file with the correct encoding
df = pd.read_csv('응답_데이터.csv', encoding='utf-8')

# Function to convert OX strings to binary responses
def ox_to_binary(ox_string):
    return [1 if char == 'O' else 0 for char in ox_string]

# Apply the function to the 'OX리스트' column
df['responses'] = df['OX리스트'].apply(ox_to_binary)

# Create a DataFrame of responses
response_data = pd.DataFrame(df['responses'].tolist())

# Convert response_data to a NumPy array
data = response_data.values

# Number of examinees and items
n_examinees, n_items = data.shape

# Build the IRT model using PyMC 4
with pm.Model() as irt_model:
    # Ability parameters for each examinee
    theta = pm.Normal('theta', mu=0, sigma=1, shape=n_examinees)
    
    # Difficulty parameters for each item
    beta = pm.Normal('beta', mu=0, sigma=1, shape=n_items)
    
    # Discrimination parameters for each item (2PL model)
    alpha = pm.HalfNormal('alpha', sigma=1, shape=n_items)
    
    # Expected probability of correct response
    p = pm.math.sigmoid(alpha * (theta[:, None] - beta))
    
    # Likelihood (observed data)
    observed = pm.Bernoulli('observed', p=p, observed=data)
    
    # Sample from the posterior distribution
    trace = pm.sample(1000, tune=1000, cores=2, random_seed=42, return_inferencedata=True)
    
# Extract estimated item parameters
alpha_est = trace.posterior['alpha'].mean(dim=['chain', 'draw']).values
beta_est = trace.posterior['beta'].mean(dim=['chain', 'draw']).values

# Extract estimated person abilities
theta_est = trace.posterior['theta'].mean(dim=['chain', 'draw']).values

# Output the results
print('Estimated Item Discrimination Parameters (alpha):')
print(alpha_est)

print('\nEstimated Item Difficulty Parameters (beta):')
print(beta_est)

print('\nEstimated Person Ability Parameters (theta):')
print(theta_est)

# IRT 분석 결과 저장

# 1. 문항 매개변수(alpha_est, beta_est)를 데이터프레임으로 저장
item_parameters = pd.DataFrame({
    '문항 번호': np.arange(1, n_items+1),
    '변별도 (alpha)': alpha_est,
    '난이도 (beta)': beta_est
})

# CSV 파일로 저장
item_parameters.to_csv('문항_매개변수.csv', index=False, encoding='utf-8-sig')

# 2. 수험생 능력 추정치(theta_est)를 데이터프레임으로 저장
# 개인정보 보호를 위해 필요한 정보만 포함하도록 합니다.
examinee_parameters = df[['이름']].copy()
examinee_parameters['능력 추정치 (theta)'] = theta_est

# CSV 파일로 저장
examinee_parameters.to_csv('수험생_능력_추정치.csv', index=False, encoding='utf-8-sig')


# trace를 NetCDF 파일로 저장
trace.to_netcdf('irt_trace.nc')


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [theta, beta, alpha]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 5 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


Estimated Item Discrimination Parameters (alpha):
[1.19434408 0.34204735 1.0193842  1.24700264 0.42140206 0.67095384
 0.90846105 1.04772355 1.04148466 1.18027999 0.51560909 0.70414866
 0.53717661 1.30900787 1.06959662 0.67446196 0.28568881 0.62479581
 0.78649851 0.41358455 0.6509522  1.12620619 1.28656096 1.02107435
 0.93286223 1.01985054 0.98872195 1.38345658 0.92883945 0.88811548
 1.18242596 0.89221827 1.11927337 1.23772874 1.12550365 1.30084682
 0.91822276 1.45438063 0.52309404 1.20768676 0.26082539 0.53399309
 1.22867614 0.88751664 1.27114632 0.38749858 0.9333516  0.71519089
 0.56046801 0.88839416]

Estimated Item Difficulty Parameters (beta):
[ 0.50504255  0.19297413  0.41704848  0.48605421  0.71461467  0.6130575
  0.82250803  0.59282018  0.23824499  0.34448233  0.13913075  0.97404142
  0.6689307   0.30328234  0.37679246  0.75142576  0.2418887   0.27433876
  0.52075568  0.01522222  0.41856188 -0.24155302  0.31752537  0.25200379
  0.45586083 -0.06408011  0.43620575  0.11549014  0.2

'irt_trace.nc'